한국어 영화 리뷰 - BERT

In [3]:
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
df

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워
...,...,...,...,...
14720,어른들을 위한 동화 정말 오랜만에 좋은 애니를 보았습니다 가족의 소중...,10,2018.01.12,코코
14721,디즈니는 못해도 본전은 한다.,7,2018.01.12,코코
14722,가족을 위한 영화... 괜찮은 영화.~~~,8,2018.01.12,코코
14723,간만에 제대로 잘짜여진 각본의 영화를 봤네 여운이 아직도 남아~어른을 위한 애니~,10,2018.01.12,코코


In [8]:
# rating 6이상이면 긍정  라벨 생성
import numpy as np
from sklearn.model_selection import train_test_split
# y를 생성해서 review컬럼이 x 데이터 분할 데이터 개수 확인
df['y'] = np.where(df['rating'] >= 6, 1, 0)
# 데이터셋을 학습 평가로 나눈다 x_train, x_val, x_test
x_, x_test, y_, y_test = train_test_split(df['review'], df['y'], test_size=0.2, random_state=42, stratify=df['y'])
x_train, x_val, y_train, y_val = train_test_split(x_, y_, test_size=0.25, random_state=42, stratify=y_)
print(f"학습 데이터: {len(x_train)}, 평가 데이터: {len(x_val)}, 테스트 데이터: {len(x_test)}")

학습 데이터: 8835, 평가 데이터: 2945, 테스트 데이터: 2945


In [12]:
import torch
# from datasets import load_metric # 2022 이후로 Huggingface datasets 라이브러리로 이전
# old system -> legacy system
import evaluate
accuracy_metric = evaluate.load("accuracy")  #  legacy system


In [13]:
def compute_metrics(eval_pred):
    '''
    Args:
        eval_pred: logits, labels를 가지고 잇는 dataset
    Returns:
        accuracy
    '''
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [19]:
# 데이터셋 생성 클래스(상속)
# __init__, __getitem__, __len__ 
# x, y      각각 텐서     y의 개수
from torch.utils.data import Dataset
class OurDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)


In [22]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased") # 다국어 BERT 토크나이저
print(tokenizer.tokenize("안녕하세요. 반갑습니다"))  # 토큰화 예시

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다']


In [25]:
input = tokenizer('안녕하세요. 반갑습니다')
input

{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [27]:
# 2022년 이후로는 Auto~~ 토크나이저와 모델을 사용하도록 권장(벤더사에서 업데이트시 유리)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
input = tokenizer('안녕하세요. 반갑습니다')
input

{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
# mBERT + Trainer 로 미세조정(Fine-Tuning) 하기
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
# 토큰화
train_encodings = tokenizer(list(x_train), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(x_val), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(x_test), truncation=True, padding=True, max_length=128)
# Dataset생성
train_dataset = OurDataset(train_encodings, list(y_train))
val_dataset = OurDataset(val_encodings, list(y_val))
test_dataset = OurDataset(test_encodings, list(y_test))
# 사전학습된 다국어 BERT 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
# 분류 모델  생성
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_strategy="epoch"            # evaluation strategy to adopt during training
)
# Trainer에 사용할 Arguments 셋팅
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics      # evaluation metrics
)
# Trainer객체생성  및 학습(미세조정)
trainer.train()
# 평가
trainer.evaluate(eval_dataset=test_dataset)  # 테스트 데이터로 평가
# 예측
predictions = trainer.predict(test_dataset)
predictions  # 예측 결과 logits, label_ids, metrics 포함

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\31799\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


AttributeError: 'list' object has no attribute 'clone'